<a href="https://colab.research.google.com/github/yendruamarendra/Delta/blob/main/delta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TASK 2

# Install Pyspark


In [1]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 45 kB/s 
     |████████████████████████████████| 200 kB 35.0 MB/s 
     |████████████████████████████████| 199 kB 21.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=bf56c0049c9cb9a18977a73d1c008b5d426f044c07e001d94ec103a6527be2b5
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


Imports

In [7]:
##from pyspark.sql import SparkSession
from pyspark.shell import spark
import pyspark.sql.functions as f
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from datetime import datetime

In [8]:
spark =  SparkSession.builder.appName("Delta_flights").getOrCreate()

# Read Flight leg Data as dataframe


In [9]:
df = spark.read.csv("/content/Delta/Dummy_Flight_Leg_Data.csv",header = True, inferSchema=True)

In [10]:
df.count()

244080

## Considering flighkey as primary filtering out records for null if there are any

In [11]:
df = df.filter(f.col("flightkey").isNotNull())

## Using window function for partition and order by

In [12]:
windowSpec = Window.partitionBy("flightkey").orderBy(f.col("flight_dt").desc(),f.col("lastupdt").desc())

## Creating Column row_number over windowSpec and selection first record to get recent status of flights

In [13]:
df_res = df.withColumn("row_number",row_number().over(windowSpec)).filter("row_number=1").drop("row_number")

## Displaying records for all the flights with most recent status

In [19]:
df_res.show()

+------------+---------+---------+---------+---------+------------+--------+-----------------+
|Carrier Code|flight_dt|flightnum|orig_arpt|dest_arpt|flightstatus|lastupdt|        flightkey|
+------------+---------+---------+---------+---------+------------+--------+-----------------+
|          DL| 01/01/19|     1038|      ATL|      ROC|    Boarding|    9:28|DL434661038ATLROC|
|          DL| 01/01/19|     1052|      ATL|      MKE|    Boarding|   14:05|DL434661052ATLMKE|
|          DL| 01/01/19|     1053|      ATL|      PBI|    Boarding|   16:04|DL434661053ATLPBI|
|          DL| 01/01/19|     1063|      ATL|      DTW|    Boarding|   11:01|DL434661063ATLDTW|
|          DL| 01/01/19|     1072|      ATL|      PIT|    Boarding|   22:17|DL434661072ATLPIT|
|          DL| 01/01/19|     1075|      ATL|      DTW|    Boarding|   12:50|DL434661075ATLDTW|
|          DL| 01/01/19|     1088|      ATL|      OKC|    Boarding|    8:20|DL434661088ATLOKC|
|          DL| 01/01/19|     1090|      ATL|      

## USING SQL - TASK 1

In [15]:
df.createOrReplaceTempView("flight")

In [18]:
spark.sql("""SELECT * FROM ( 
  SELECT *, ROW_NUMBER() 
  OVER(PARTITION BY flightkey ORDER BY flight_dt desc,lastupdt DESC) AS row_number 
  FROM flight ) temp 
WHERE Row_number=1""").drop("row_number").show()

+------------+---------+---------+---------+---------+------------+--------+-----------------+
|Carrier Code|flight_dt|flightnum|orig_arpt|dest_arpt|flightstatus|lastupdt|        flightkey|
+------------+---------+---------+---------+---------+------------+--------+-----------------+
|          DL| 01/01/19|     1038|      ATL|      ROC|    Boarding|    9:28|DL434661038ATLROC|
|          DL| 01/01/19|     1052|      ATL|      MKE|    Boarding|   14:05|DL434661052ATLMKE|
|          DL| 01/01/19|     1053|      ATL|      PBI|    Boarding|   16:04|DL434661053ATLPBI|
|          DL| 01/01/19|     1063|      ATL|      DTW|    Boarding|   11:01|DL434661063ATLDTW|
|          DL| 01/01/19|     1072|      ATL|      PIT|    Boarding|   22:17|DL434661072ATLPIT|
|          DL| 01/01/19|     1075|      ATL|      DTW|    Boarding|   12:50|DL434661075ATLDTW|
|          DL| 01/01/19|     1088|      ATL|      OKC|    Boarding|    8:20|DL434661088ATLOKC|
|          DL| 01/01/19|     1090|      ATL|      